# dlib 라이브러리로 Face Recognition 수행하기

dlib은 얼굴 관련 기본 예측모델과 응용모델을 제공해주는 고마운 라이브러리입니다. 

아래 예시는 얼굴의 눈코입, 눈썹 턱선 등 랜드마크 포인트 68개의 점을

추출하는 예시입니다. ZOOM의 눈썹, 입술 메이크업이나

카메라효과로 유명한 SNOW 어플 등에서 많이 활용하는 기능이고,

눈을 감았는지 떴는지, 입을 벌렸는지 등을 판단할 수 있기 때문에

운전자 졸음예측(drowsiness detection),

얼굴 반듯하게 정렬(face alignment) 및

신원인식 등 다양한 분야에 landmarking이 사용됩니다.

![](https://i.ibb.co/yNrsv4b/landmarks.png)

아래 이미지에서 랜드마크를 추출해봅시다.

![](https://i.ibb.co/ZdyyTBT/face.jpg)


In [3]:
# 전체 예측모델 다운로드/압축해제 후 취사선택 가능
!git clone https://github.com/davisking/dlib-models.git

Cloning into 'dlib-models'...
Updating files:  84% (16/19)
Updating files:  89% (17/19)
Updating files:  94% (18/19)
Updating files: 100% (19/19)
Updating files: 100% (19/19), done.


In [2]:
# 단, 위 레포지토리는 용량이 꽤 크고 
# 정리하기도 까다롭습니다.

# 아래 예제에서 사용할 
# 얼굴 랜드마크 포인트 97개 추출용 dat파일(99.7MB)
# 하나만 다운받고 싶으시면

%pip install gdown

import gdown
gdown.download('https://drive.google.com/uc?id=13pN7jYwdpNqfWHPG-tg2Cv31xtiekb2t')

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip
Downloading...
From: https://drive.google.com/uc?id=13pN7jYwdpNqfWHPG-tg2Cv31xtiekb2t
To: C:\Users\Administrator\PycharmProjects\jss_tutorials\3_dlib을 활용한얼굴인식\shape_predictor_68_face_landmarks.dat
100%|██████████| 99.7M/99.7M [00:03<00:00, 27.5MB/s]


'shape_predictor_68_face_landmarks.dat'

In [ ]:
# dlib 라이브러리는 바로 pip로 설치하지 말고,
# 아래 링크를 참고해서 설치해주세요.
# https://m.blog.naver.com/chandong83/222459722192

# 여러가지 다른 방법(소스코드를 다운받아 직접 컴파일 등)이
# 있지만, 위 방법이 가장 간단합니다.

# 리눅스에서는 간단하게 `pip install dlib`으로 
# 설치하실 수 있습니다. 

In [1]:
from urllib.request import urlretrieve
import numpy as np
import dlib
import cv2
import gdown

In [2]:
# 68개의 facial landmark 정의
RIGHT_EYE = list(range(36, 42))
LEFT_EYE = list(range(42, 48))
EYES = list(range(36, 48))
MOUTH = list(range(48, 68))
NOSE = list(range(27, 36))
EYEBROWS = list(range(17, 27))
JAWLINE = list(range(1, 17))
ALL = list(range(0, 68))

In [8]:
urlretrieve("https://i.ibb.co/ZdyyTBT/face.jpg")
image_file = './face2.jpg'

gdown.download("https://drive.google.com/uc?id=13pN7jYwdpNqfWHPG-tg2Cv31xtiekb2t")
predictor_file = "./shape_predictor_68_face_landmarks.dat"

Downloading...
From: https://drive.google.com/uc?id=13pN7jYwdpNqfWHPG-tg2Cv31xtiekb2t
To: C:\Users\Administrator\PycharmProjects\jss_tutorials\3_dlib을 활용한얼굴인식\shape_predictor_68_face_landmarks.dat
100%|██████████| 99.7M/99.7M [00:05<00:00, 19.0MB/s]


In [10]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(predictor_file)

In [11]:
image = cv2.imread(image_file)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

In [12]:
rects = detector(gray, 1)
print(f"Number of faces detected: {len(rects)}")

Number of faces detected: 1


In [ ]:
for i, rect in enumerate(rects):
    points = np.matrix([[p.x, p.y] for p in predictor(gray, rect).parts()])
    show_parts = points[ALL]
    
    for j, point in enumerate(show_parts):
        x = point[0,0]
        y = point[0,1]
        cv2.circle(image, (x, y), 5, (0, 255, 255), -1)
        cv2.putText(image, f"{j + 1}", (x, y-2), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

cv2.imshow("final image", image)
cv2.waitKey(0)  # 아무 키나 누르면 종료됨
cv2.destroyAllWindows()
cv2.imwrite("face_output.jpg", image)

![](https://i.ibb.co/R95NT3N/face-output.jpg)


이 모델은 얼굴이 두 개 이상이라도 문제없이 예측합니다.

그런데 이상한 점이 하나 있습니다.

아래 사진의 인물은 한 명도 예측하지 못합니다.

아쉽지만 정면으로 반듯한 얼굴로 훈련을 많이 했기 때문에

90도 가까이 회전되어 있는 얼굴에서는

랜드마크를 추출하기가 쉽지 않습니다.

(훈련데이터가 부족한 탓이기도 합니다.)

![](https://i.ibb.co/VJ5QWK6/faces.jpg)


In [126]:
urlretrieve("https://i.ibb.co/VJ5QWK6/faces.jpg")
image_file = './faces.jpg'
image = cv2.imread(image_file)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
rects = detector(gray, 1)
print(f"Number of faces detected: {len(rects)}")
for i, rect in enumerate(rects):
    points = np.matrix([[p.x, p.y] for p in predictor(gray, rect).parts()])
    show_parts = points[ALL]
    
    for j, point in enumerate(show_parts):
        x = point[0,0]
        y = point[0,1]
        cv2.circle(image, (x, y), 5, (0, 255, 255), -1)
        cv2.putText(image, f"{j + 1}", (x, y-2), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

cv2.imshow("final image", image)
cv2.waitKey(0)  # 아무 키나 누르면 종료됨
cv2.destroyAllWindows()
cv2.imwrite("face2_output.jpg", image)

Number of faces detected: 0


True

0명으로 예측하네요. 회전해보면 어떨까요?



In [127]:
image_file = './faces_l.jpg'
image = cv2.imread(image_file)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
rects = detector(gray, 1)
print(f"Number of faces detected: {len(rects)}")
for i, rect in enumerate(rects):
    points = np.matrix([[p.x, p.y] for p in predictor(gray, rect).parts()])
    show_parts = points[ALL]
    
    for j, point in enumerate(show_parts):
        x = point[0,0]
        y = point[0,1]
        cv2.circle(image, (x, y), 5, (0, 255, 255), -1)
        cv2.putText(image, f"{j + 1}", (x, y-2), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

cv2.imshow("final image", image)
cv2.waitKey(0)  # 아무 키나 누르면 종료됨
cv2.destroyAllWindows()
cv2.imwrite("faces_l_output.jpg", image)

Number of faces detected: 2


True

![](https://i.ibb.co/xLvmsKt/faces-l-output.jpg)

얼굴이 반듯해진 두 사람만 잘 예측해주네요^^

편리하게 쓸 수 있는 기본모델이지만 아쉬운 부분이 있긴 하네요.

아래는 우리나라 가수 부석순입니다.

![](https://i.ibb.co/LxKbftS/bss.jpg)

In [128]:
urlretrieve("https://i.ibb.co/LxKbftS/bss.jpg")
image_file = './bss.jpg'  # 파일명에 한글을 쓰면 오류가 남(부석순)
image = cv2.imread(image_file)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
rects = detector(gray, 1)
print(f"Number of faces detected: {len(rects)}")
for i, rect in enumerate(rects):
    points = np.matrix([[p.x, p.y] for p in predictor(gray, rect).parts()])
    show_parts = points[ALL]
    
    for j, point in enumerate(show_parts):
        x = point[0,0]
        y = point[0,1]
        cv2.circle(image, (x, y), 1, (0, 255, 255), -1)
        cv2.putText(image, f"{j + 1}", (x, y-2), cv2.FONT_HERSHEY_SIMPLEX, 0.2, (0, 255, 0), 1)

cv2.imshow("final image", image)
cv2.waitKey(0)  # 아무 키나 누르면 종료됨
cv2.destroyAllWindows()
cv2.imwrite("./bss_output.jpg", image)

Number of faces detected: 3


True


![](https://i.ibb.co/j4bgCFL/bss-output.jpg)

여러분의 사진, 여러분이 가지고 있는 얼굴 사진으로도 한 번 예측해보시기 바랍니다.

다음 챕터에서는 동영상 랜드마크 추출 후에, 졸읍분석을 해 보겠습니다.